In [ ]:
import pandas as pd
df = pd.read_csv("C:/Users/Salif SAWADOGO/OneDrive - EQUATORIAL COCA-COLA BOTTLING COMPANY S.L/dynamic segmentation/matching/output/training_set_matching.csv")
df

In [ ]:
df.apply(lambda x: print(x), axis=1)

In [ ]:
df["index"] = df.apply(lambda x: "{}_{}".format(x["L_CUSTOMER_ID"], x["R_id"]), axis=1)
df["dist_similarity"] = (1000-df["dist_m"])/1000
df[["index", "dist_similarity", "name_similarity", "address_similarity", "match_status"]]

In [ ]:
from sklearn.linear_model import LogisticRegression
y = df["match_status"]
X = df[["name_similarity", "address_similarity"]]
clf = LogisticRegression(random_state=0).fit(X, y)
clf.coef_

In [ ]:
sdf = spark.createDataFrame(df)
sdf.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as F
lr = LogisticRegression()

In [ ]:
training = sdf[["index", "dist_similarity", "name_similarity", "address_similarity", "match_status"]].withColumn("label", F.col("match_status"))
assembler = VectorAssembler(inputCols=["name_similarity", "address_similarity"], outputCol="features")
training_ready = assembler.transform(training)
training_ready.show()

In [ ]:
model_fit = lr.fit(training_ready)
model_fit.coefficients / sum(model_fit.coefficients)